In [1]:
import os
import time
import math

import pandas as pd
import torch

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModel
from pinecone import Pinecone, ServerlessSpec

In [2]:
# Load data
data = pd.read_csv('../data/judgment.csv', skipinitialspace=True)
data[data['CASE_ID']==70265]

,CASE_ID,CASENAME,CASENUMBER,DATE,COURTNAME,CASETYPE,JUDGETYPE,CASEURL,DECISION,SUBSTANCE,REFERENCE,CONTENT,WINSTATUS
70,70265,근로기준법위반,2005노818,2005-11-18,울산지방법원,형사,판결 : 상고,/DRF/lawService.do?OC=wkdtmf357&target=prec&ID...,[1] 근로기준법 제8조에 정한 ‘영리로’의 의미 및 영리성 유무...,[1] 근로기준법 제8조(중간착취의 배제)는 누구든지 법률에 의하...,"[1] 근로기준법 제8조, 제110조 / [2] ...",【피 고 인】 피고인【항 소 인】 피고인【검 사】 전미화【변 호 인】 변호사 ...,DEFENDANT


In [3]:
# Remove unnecessary columns
data = data[['CASE_ID', 'CONTENT']]
data = data.rename(columns={'CASE_ID': 'ID', 'CONTENT': 'facts'})
data.head()

,ID,facts
0,64515,"【항고인, 행위자】 【원심결정】 부산지방법원 가정지원 2008. 5. 9.자 200..."
1,64516,【준항고인】 【주 문】인천지방검찰청 소속 사법경찰관 \n 청구외 ...
2,64517,【피 고 인】 【항 소 인】 피고인들【검 사】 박동진【변 호 인】 법무법인 태...
3,64518,【피 고 인】 【항 소 인】 피고인【검 사】 이원규【변 호 인】 변호사 하창우...
4,64519,【피 고 인】 【항 소 인】 피고인【검 사】 박성재【변 호 인】 법무법인 세인...


In [7]:
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.eval()

embeddings = []
for _, row in tqdm(data.iterrows(), desc='Calculating Embeddings...', total=len(data)):
    facts = row['facts'].replace('\n', ' ')  
    encoded_input = tokenizer(facts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        embedding = model_output.pooler_output[0].cpu().detach().numpy()
    
    embeddings.append(embedding)

# 새로운 열로 추가
data['embeddings'] = embeddings
data.head()

Calculating Embeddings...:   0%|          | 0/6083 [00:00<?, ?it/s]

,ID,facts,embeddings
0,64515,"【항고인, 행위자】 【원심결정】 부산지방법원 가정지원 2008. 5. 9.자 200...","[-0.056413822, 0.013409576, -0.61507624, 0.039..."
1,64516,【준항고인】 【주 문】인천지방검찰청 소속 사법경찰관 \n 청구외 ...,"[-0.11401074, 0.0027919414, -0.5784367, -0.021..."
2,64517,【피 고 인】 【항 소 인】 피고인들【검 사】 박동진【변 호 인】 법무법인 태...,"[-0.10487191, -0.013258798, -0.6071003, -0.023..."
3,64518,【피 고 인】 【항 소 인】 피고인【검 사】 이원규【변 호 인】 변호사 하창우...,"[-0.07967803, -0.04007791, -0.59973556, -0.002..."
4,64519,【피 고 인】 【항 소 인】 피고인【검 사】 박성재【변 호 인】 법무법인 세인...,"[-0.102022685, -0.029293647, -0.5323853, 0.012..."


In [8]:
PC_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PC_API_KEY)

In [19]:
# Create a serverless index
index_name = 'courtcase'

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
    

In [20]:
index = pc.Index(index_name)

# Vector will be stored in pinecone DB
vectors = []
for row in tqdm(data.itertuples(index=False), desc='Creating vectors...', total=len(data)):
    vectors.append({
        "id": str(row.ID),
        "values": row.embeddings.tolist()
    })
    
# Pinecone은 요청당 업로드할 수 있는 데이터 크기를 4MB로 제한하므로 batch로 잘라서 업로드
# 또한 Pinecone은 metadata의 크기가 40KB로 제한되므로 원본 텍스트는 MySQL에 저장
batch_size = 100
# 벡터를 나누어 업로드
for i in tqdm(range(math.ceil(len(vectors) / batch_size)), desc="Uploading vectors..."):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(vectors))
    batch = vectors[start_idx:end_idx]
    
    # Pinecone에 업로드
    index.upsert(
        vectors=batch,
        namespace="ns1"
    )

Creating vectors...:   0%|          | 0/6083 [00:00<?, ?it/s]

Uploading vectors...:   0%|          | 0/61 [00:00<?, ?it/s]

In [21]:
print(index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 6083}},
 'total_vector_count': 6083}


In [18]:
index_name = 'courtcase'
pc.delete_index(index_name)